In [ ]:
import os
from copy import deepcopy

import xarray as xr
import numpy as np
import pandas as pd
import scipy.stats
import scipy.ndimage
from tqdm import tqdm_notebook as tqdm

import distributed

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
import matplotlib.colors as mplc
import cartopy.crs as ccrs
import matplotlib.dates as mdates
import matplotlib.lines as mpll
from pandas.plotting import register_matplotlib_converters

In [ ]:
rnd = np.random.RandomState(42)

In [ ]:
plt.style.use('paper')
plt.style.use('egu_journals')
register_matplotlib_converters()
rotated_pole = ccrs.RotatedPole(pole_longitude=-171.0, pole_latitude=41.5)
plate_carree = ccrs.PlateCarree()

In [ ]:
cluster = distributed.LocalCluster()
client = distributed.Client(cluster)
client

# Load data

In [ ]:
base_path ='/work/um0203/u300636/for2131/runs/da_enkf_for_soil/'

## H2O

In [ ]:
vr_h2o_path = os.path.join(base_path, '016', 'h2o_cleaned.nc')
vr_h2o = xr.open_dataset(vr_h2o_path)['H2OSOI'].squeeze(drop=True).isel(levsoi=4).chunk((1, 302, 267))[..., 30:-30, 30:-30]

In [ ]:
det_h2o_path = os.path.join(base_path, '018', 'h2o_cleaned.nc')
det_h2o = xr.open_dataset(det_h2o_path)['H2OSOI'].squeeze(drop=True).isel(levsoi=4).chunk((1, 302, 267))[..., 30:-30, 30:-30]

In [ ]:
ens_h2o_path = os.path.join(base_path, '015', 'h2o_cleaned.nc')
ens_h2o = xr.open_dataset(ens_h2o_path)['H2OSOI'].squeeze(drop=True).isel(levsoi=4).chunk((40, 1, 302, 267))[..., 30:-30, 30:-30]

In [ ]:
da_off_h2o_path = os.path.join(base_path, '019', 'h2o_cleaned.nc')
da_off_h2o = xr.open_dataset(da_off_h2o_path)['H2OSOI'].squeeze(drop=True).isel(levsoi=4).chunk((40, 1, 302, 267))[..., 30:-30, 30:-30]

In [ ]:
da_h2o_path = os.path.join(base_path, '020', 'h2o_cleaned.nc')
da_h2o = xr.open_dataset(da_h2o_path)['H2OSOI'].squeeze(drop=True).isel(levsoi=4).chunk((40, 1, 302, 267))[..., 30:-30, 30:-30]

In [ ]:
sekf_h2o_path = os.path.join(base_path, '023', 'juwels', 'h2o_cleaned.nc')
sekf_h2o = xr.open_dataset(sekf_h2o_path)['H2OSOI'].squeeze(drop=True).isel(levsoi=4).chunk((1, 302, 267))[..., 30:-30, 30:-30]

In [ ]:
da_off_h2o['ensemble'] = ens_h2o['ensemble'] = da_h2o['ensemble'] = np.arange(40)

# Error estimation

In [ ]:
err_ens_h2o = np.sqrt(((ens_h2o.mean('ensemble')-vr_h2o) ** 2).mean(['lat', 'lon'])).load()
err_da_off_h2o = np.sqrt(((da_off_h2o.mean('ensemble')-vr_h2o) ** 2).mean(['lat', 'lon'])).load()
err_da_h2o = np.sqrt(((da_h2o.mean('ensemble')-vr_h2o) ** 2).mean(['lat', 'lon'])).load()
err_det_h2o = np.sqrt(((det_h2o-vr_h2o) ** 2).mean(['lat', 'lon'])).load()
err_sekf_h2o = np.sqrt(((sekf_h2o-vr_h2o) ** 2).mean(['lat', 'lon'])).load()

In [ ]:
ens_spread_da_h2o = (np.sqrt(da_h2o.var('ensemble', ddof=1).mean(['lat', 'lon'])) * np.sqrt(41/40)).load()
ens_spread_ens_h2o = (np.sqrt(ens_h2o.var('ensemble', ddof=1).mean(['lat', 'lon'])) * np.sqrt(41/40)).load()
ens_spread_da_off_h2o = (np.sqrt(da_off_h2o.var('ensemble', ddof=1).mean(['lat', 'lon'])) * np.sqrt(41/40)).load()

# Plot

In [ ]:
plt.rcParams['text.usetex'] = False

In [ ]:
fig, ax = plt.subplots()
#ax.axhline(y=0, c='black')
#ax.axvline(x=pd.to_datetime('2015-07-31 12:00'), c='0.5', alpha=0.5)
ax.fill_betweenx(x1=[pd.to_datetime('2015-07-31 12:00')]*2, y=[0, 1], color='0.8', alpha=0.35, lw=0)
ax.axvline(x=pd.to_datetime('2015-07-30 00:00'), c='black', lw=0.5, zorder=-99)
ax.axvline(x=pd.to_datetime('2015-07-31 00:00'), c='black', lw=0.5, zorder=-99)
ax.axvline(x=pd.to_datetime('2015-08-01 00:00'), c='black', lw=0.5, zorder=-99)
ax.axvline(x=pd.to_datetime('2015-08-02 00:00'), c='black', lw=0.5, zorder=-99)
ax.axvline(x=pd.to_datetime('2015-08-03 00:00'), c='black', lw=0.5, zorder=-99)
ax.axvline(x=pd.to_datetime('2015-08-04 00:00'), c='black', lw=0.5, zorder=-99)
ax.axvline(x=pd.to_datetime('2015-08-05 00:00'), c='black', lw=0.5, zorder=-99)
ax.axvline(x=pd.to_datetime('2015-08-06 00:00'), c='black', lw=0.5, zorder=-99)
ax.axvline(x=pd.to_datetime('2015-08-07 00:00'), c='black', lw=0.5, zorder=-99)

ax.plot(err_ens_h2o.time, err_ens_h2o, label='RMSE ENS', c='black')
ax.plot(ens_spread_ens_h2o.time, ens_spread_ens_h2o, c='black', ls='--', label='Spread ENS')
ax.plot(err_det_h2o.time, err_det_h2o, label='RMSE DET', c='C0')
ax.plot(err_sekf_h2o.time, err_sekf_h2o, label='RMSE SEKF', c='deepskyblue')
ax.plot(err_da_off_h2o.time, err_da_off_h2o, label='RMSE LETKF Soil', c='salmon')
ax.plot(ens_spread_da_off_h2o.time, ens_spread_da_off_h2o, label='Spread LETKF Soil', c='salmon', ls='--')
ax.plot(err_da_h2o.time, err_da_h2o, label='RMSE LETKF Soil+Temp', c='firebrick')
ax.plot(ens_spread_da_h2o.time, ens_spread_da_h2o, label='Spread LETKF Soil+Temp', c='firebrick', ls='--')

ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
ax.set_xlim(pd.to_datetime('2015-07-29 18:00'), pd.to_datetime('2015-08-08 00:00'))
ax.set_xlabel('Time (UTC)')
ax.set_ylabel('Error for soil moisture in root-depth (m$^3$/m$^3$)')
ax.legend(loc=0, framealpha=1)
ax.set_ylim(0, 0.023)
#ax.plot(spread_t2m.time, spread_t2m, label='Ens Spread', c='firebrick')
plt.show()
fig.savefig('../figures/fig_05_err_h2o_long.png', dpi=300)